In [1]:
# Load Turing.
using Turing

# Load CSV, DataFrames
using CSV, DataFrames

# Load StatsPlots for visualizations and diagnostics.
using StatsPlots

# Functionality for splitting and normalizing the data.
using MLDataUtils: shuffleobs, splitobs, rescale!

# We need a softmax function which is provided by NNlib.
using NNlib: softmax

# Functionality for constructing arrays with identical elements efficiently.
using FillArrays

# Functionality for working with scaled identity matrices.
using LinearAlgebra

# Set a seed for reproducibility.
using Random

# For save
using JLD

using ReverseDiff

using Memoization

using Optim, StatsBase

Random.seed!(0)

TaskLocalRNG()

In [7]:
# Import the dataset.
data = DataFrame(CSV.File("dataForTuring//E4orth.csv"))
# Show twenty random rows.
data[rand(1:size(data, 1), 20), :]

Row,Column1,subj_id,test_part,word,letter,mask_A,mask_B,mask_C,mask_D,mask_E,mask_F,mask_G,mask_H,mask_I,mask_J,mask_K,mask_L,mask_M,mask_N,mask_O,mask_P,mask_Q,mask_R,mask_S,mask_T,mask_U,mask_V,mask_W,mask_X,mask_Y,mask_Z,post_A,post_B,post_C,post_D,post_E,post_F,post_G,post_H,post_I,post_J,post_K,post_L,post_M,post_N,post_O,post_P,post_Q,post_R,post_S,post_T,post_U,post_V,post_W,post_X,post_Y,post_Z,hit_bin,prior_A,prior_B,prior_C,prior_D,prior_E,prior_F,prior_G,prior_H,prior_I,prior_J,prior_K,prior_L,prior_M,prior_N,prior_O,prior_P,prior_Q,prior_R,prior_S,prior_T,prior_U,prior_V,prior_W,prior_X,prior_Y,prior_Z,eig_A,eig_B,eig_C,eig_D,eig_E,eig_F,eig_G,eig_H,eig_I,eig_J,eig_K,eig_L,eig_M,eig_N,eig_O,eig_P,⋯
,Int64,String15,String15,String15,String1,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,19738,375d9eb,nonpretend,dalai lama,A,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2.06114,-1.3453,-0.216165,0.484639,1.74338,-0.591815,0.171802,0.201978,0.947121,-0.269125,-0.00678563,0.937268,-0.00432235,1.96815,0.86768,-0.216165,-1.04752,0.726041,0.423673,0.457543,-0.318391,-0.751929,-0.597974,-0.813511,0.362707,-0.862777,true,-0.15156,-2.09311,0.355215,0.148453,1.36212,0.546431,-0.190891,0.896313,0.487563,-0.316423,-0.375168,-0.306654,0.0687078,-0.457902,0.704187,0.122382,0.343378,0.41223,0.767485,1.48299,0.437394,0.327831,0.571429,0.155273,-0.367784,0.17729,1.22701,-1.2671,1.6882,1.70004,1.34366,1.57894,1.67757,1.67215,1.62711,1.63807,1.66032,1.64097,1.72687,1.26097,1.60277,1.64633,⋯
2,35829,568629e6,nonpretend,tooth,H,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.730018,0.725541,-1.27525,-0.568648,1.54753,-0.761794,-0.249598,0.966766,0.823682,-0.761794,0.0668177,1.18276,-0.0855374,1.19298,0.998748,-0.422942,-0.761794,1.06623,0.826258,1.72334,0.698673,-0.444268,-0.0222716,-0.761794,-0.592368,-0.761794,true,1.07565,-0.762019,-1.8409,1.19387,1.5727,0.829882,0.281985,0.285226,0.657846,0.240782,-0.352926,-0.484948,0.238504,0.257245,0.645601,0.418188,0.224778,0.16393,0.479154,0.404476,-0.387909,0.17546,0.16744,0.239954,0.586923,0.218983,1.63269,1.61952,-1.18599,1.68532,1.41408,1.57053,1.69137,1.55488,1.70135,1.55018,1.71629,1.52002,1.67436,1.55373,1.594,1.70004,⋯
3,41445,995d7ff,pretend,tooth,T,0,1,1,1,0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,-1.30374,0.0133245,0.0133245,0.0133245,-1.30374,0.0133245,0.0133245,-1.30374,-1.30374,0.0133245,0.0133245,0.0133245,0.0133245,0.0133245,-1.30374,0.0133245,0.0133245,0.0133245,0.0133245,4.35635,0.0133245,0.0133245,0.0133245,0.0133245,0.0133245,0.0133245,true,-2.06019,0.16185,0.504184,0.894552,-2.06019,0.373089,0.300641,-2.06019,-2.06019,-0.193488,-0.0292207,0.834046,0.404403,1.55693,-2.06019,0.277819,-0.20888,1.35471,1.44494,0.398111,0.497815,0.0254467,0.392196,-0.194284,0.289761,-0.214453,-1.20452,-0.500657,-0.336203,-0.148675,-1.20452,-0.39918,-0.433983,-1.20452,-1.20452,-0.671357,-0.592445,-0.177741,-0.384137,0.169523,-1.20452,-0.444946,⋯
4,3343,14560a55,pretend,dalai lama,E,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1.24586,-0.150638,-0.150638,2.13407,-0.150638,-0.150638,-0.150638,-0.150638,2.13407,-0.150638,-0.150638,2.13407,2.13407,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,-0.150638,false,-1.92078,0.0259308,0.2243,0.314659,1.74974,0.1483

In [8]:
# Recode columns
letters = map(string, collect('A':'Z'))
subjects = unique(data[:,:subj_id])
conditions = ["pretend", "nonpretend"]
data[!, :letter_index] = indexin(data[!, :letter], letters)
data[!, :subj_index] = indexin(data[!, :subj_id], subjects)
data[!, :pretend] = map(x -> x=="pretend" ? 1 : 0, data[:, :test_part])

# Show twenty random rows of the new dataframe
data[rand(1:size(data, 1), 20), [:subj_id, :subj_index, :letter, :letter_index, :test_part, :pretend]]

Row,subj_id,subj_index,letter,letter_index,test_part,pretend
,String15,Union…,String1,Union…,String15,Int64
1,4615914b,322,T,20,nonpretend,0
2,8860f4e,490,A,1,nonpretend,0
3,27461098,158,N,14,nonpretend,0
4,535e2b0,392,V,22,nonpretend,0
5,199629fa,88,Y,25,pretend,1
6,55558a43,413,E,5,nonpretend,0
7,2515d445,136,L,12,nonpretend,0
8,14960f6e,46,D,4,pretend,1
9,48259ff8,344,T,20,pretend,1


In [13]:
subjects[79]

"1885a9f5"

In [9]:
prior_cols = [Symbol("prior_", i) for i in 'A':'Z']
post_cols = [Symbol("post_", i) for i in 'A':'Z']
eig_cols = [Symbol("eig_", i) for i in 'A':'Z']
mask_cols = [Symbol("mask_", i) for i in 'A':'Z']
subj_col = :subj_index
pretend_col = :pretend
target_col = :letter_index

# Turing requires data in matrix and vector form.
priors = Matrix(data[!, prior_cols])
posts = Matrix(data[!, post_cols])
eig = Matrix(data[!, eig_cols])
masks = Matrix(data[!, mask_cols])
subj = data[!, subj_col]
pretend = data[!, pretend_col]
target = data[!, target_col]

41445-element Vector{Union{Nothing, Int64}}:
  1
  3
  4
  5
  9
 10
 11
 12
 13
 15
 18
 19
 20
  ⋮
  5
  9
 13
 14
 15
 20
  1
  5
  8
  9
 15
 20

In [10]:
# Bayesian multinomial logistic regression
@model function logistic_regression(priors, posts, eig, masks, pretend, y)
    
    n = size(priors, 1) # number of rows
    length(y) == size(posts,1) == size(masks,1) == n ||
        throw(DimensionMismatch("number of observations in `as` and 'bs' `y` is not equal"))

    coef_prior ~ Normal(0, 10)
    coef_post ~ Normal(0, 10)
    coef_eig ~ Normal(0, 10)

    # Subject-level parameters: condition effets
    coef_prior_delta ~ Normal(0, 10)
    coef_post_delta ~ Normal(0, 10)
    coef_eig_delta ~ Normal(0, 10)
    
    for i in 1:n
        v_unmasked = softmax((coef_prior*priors[i,:]+coef_post*posts[i,:]+coef_eig*eig[i,:]) +
            pretend[i]*(coef_prior_delta*priors[i,:]+coef_post_delta*posts[i,:]+coef_eig_delta*eig[i,:]))
        v = v_unmasked.*masks[i,:]
        if sum(v)>0
            v=v/sum(v)
        else 
            v = masks[i,:]/sum(masks[i,:]) # if everything sum to 0, just choose randomly from the possibilities
        end
        y[i] ~ Categorical(v)
    end
end;

In [11]:
"""
    outfun(m, outfn="output.csv")

output the coefficient table of a fitted model to a file
"""
outfun = function(m, outfn="output.csv")
    ct = coeftable(m)
    coef_df = DataFrame(ct.cols, :auto);
    rename!(coef_df, ct.colnms, makeunique = true)
    coef_df[!, :term] = ct.rownms;
    CSV.write(outfn, coef_df);
end

for i = 1:length(unique(subj))
    i
    m = logistic_regression(priors[(subj.==i),:], posts[(subj.==i),:], eig[(subj.==i),:], masks[(subj.==i),:], pretend[(subj.==i)], target[(subj.==i)])
    mle1 = optimize(m, MLE())
    outfun(mle1, "model_fits//E4//subj$(i)_orth.csv")
end

# t = @time chain = sample(m, HMC(0.05, 10), MCMCThreads(), 1_500, 3)
# t = @time chain = sample(m, Prior(), 1500)

# Save the chain to a file
# save("my_chain.jld", "chain", chain)
# CSV.write("my_chains.csv", DataFrame(chain))

┌ Warning: Optimization did not converge! You may need to correct your model or adjust the Optim parameters.
└ @ Turing C:\Users\tanzor\.julia\packages\Turing\Suzsv\src\modes\OptimInterface.jl:243
┌ Warning: Optimization did not converge! You may need to correct your model or adjust the Optim parameters.
└ @ Turing C:\Users\tanzor\.julia\packages\Turing\Suzsv\src\modes\OptimInterface.jl:243
┌ Warning: Optimization did not converge! You may need to correct your model or adjust the Optim parameters.
└ @ Turing C:\Users\tanzor\.julia\packages\Turing\Suzsv\src\modes\OptimInterface.jl:243
┌ Warning: Optimization did not converge! You may need to correct your model or adjust the Optim parameters.
└ @ Turing C:\Users\tanzor\.julia\packages\Turing\Suzsv\src\modes\OptimInterface.jl:243
